# Data Collection

With
- ## MySQL database
- ## Rest API

In [14]:
import pymysql
import pandas as pd

In [163]:
###########

Writing .env


In [164]:
#!pip install python-dotenv
import os
from dotenv import load_dotenv

load_dotenv()

True

### MySQL database
- PyMySQL - https://pymysql.readthedocs.io/en/latest/user/examples.html

In [165]:
class Config:
    MYSQL_HOST = os.getenv("MYSQL_HOST")
    MYSQL_PORT = int(os.getenv("MYSQL_PORT"))
    MYSQL_USER = os.getenv("MYSQL_USER")
    MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
    MYSQL_DB = os.getenv("MYSQL_DB")
    MYSQL_CHARSET = os.getenv("MYSQL_CHARSET")

In [167]:
os.getenv("MYSQL_PORT")

'3306'

In [38]:
print(Config.MYSQL_PORT)

3306


In [41]:
# Connect to the database
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
    cursor.execute('select * from audible_data;')
    result = cursor.fetchall()
    
print('number of row: ', len(result))
print('type: ',type(result))

number of row:  2269
type:  <class 'list'>


In [29]:
with connection.cursor() as cursor:
    cursor.execute('show tables;')
    tables = cursor.fetchall()
print(tables)

[{'Tables_in_r2de2': 'audible_data'}, {'Tables_in_r2de2': 'audible_transaction'}]


In [34]:
result[0]

{'Book_ID': 1,
 'Book Title': 'Bamboozled by Jesus',
 'Book Subtitle': 'How God Tricked Me into the Life of My Dreams',
 'Book Author': 'Yvonne Orji',
 'Book Narrator': 'Yvonne Orji',
 'Audio Runtime': '6 hrs and 31 mins',
 'Audiobook_Type': 'Unabridged Audiobook',
 'Categories': 'Biographies & Memoirs',
 'Rating': '5',
 'Total No. of Ratings': 47,
 'Price': '$29.65 '}

In [35]:
# check easier
audible_data = pd.DataFrame(result)
audible_data = audible_data.set_index('Book_ID')
audible_data.head()

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
Book_ID,,,,,,,,,,
1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65


In [27]:
audible_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2269 entries, 1 to 2269
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Book Title            2269 non-null   object 
 1   Book Subtitle         1471 non-null   object 
 2   Book Author           2269 non-null   object 
 3   Book Narrator         2211 non-null   object 
 4   Audio Runtime         2269 non-null   object 
 5   Audiobook_Type        2206 non-null   object 
 6   Categories            2152 non-null   object 
 7   Rating                2033 non-null   object 
 8   Total No. of Ratings  1975 non-null   float64
 9   Price                 2269 non-null   object 
dtypes: float64(1), object(9)
memory usage: 195.0+ KB


Join table: between fact and dimension table
- audible_transaction: `book_id`
- audible_data: `Book_ID`

In [42]:
sql = "SELECT * FROM audible_transaction"
audible_transaction = pd.read_sql(sql, connection)
audible_transaction.head()

C:\Users\Kiddy\AppData\Local\Temp\ipykernel_3312\487031142.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  audible_transaction = pd.read_sql(sql, connection)


,timestamp,user_id,book_id,country
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal
1,2021-05-01 00:00:03,561b26c1,829,United States of America
2,2021-05-01 00:00:04,81f149e5,1391,Japan
3,2021-05-01 00:00:07,4f218413,1586,Taiwan
4,2021-05-01 00:00:18,a4066781,300,United States of America


In [121]:
transaction = audible_transaction.merge(audible_data, how="left", left_on="book_id", right_on="Book_ID")
transaction.head()

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49


In [44]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998821 entries, 0 to 1998820
Data columns (total 14 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   user_id               object        
 2   book_id               int64         
 3   country               object        
 4   Book Title            object        
 5   Book Subtitle         object        
 6   Book Author           object        
 7   Book Narrator         object        
 8   Audio Runtime         object        
 9   Audiobook_Type        object        
 10  Categories            object        
 11  Rating                object        
 12  Total No. of Ratings  float64       
 13  Price                 object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(11)
memory usage: 228.7+ MB


 ----------------------------------------

### Rest API
- Requests - https://requests.readthedocs.io/en/latest/
- HTTP Status Codes - https://httpstatuses.io/

In [63]:
import requests
url = "https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate"
response = requests.get(url) # , auth=('user', 'pass')) #not have
response.status_code # check status - if 200 is OK

200

In [72]:
r.text # can use just 'r'

'{"conversion_rate":{"2021-08-12":33.084,"2021-08-11":33.145,"2021-08-10":33.46,"2021-08-09":33.464,"2021-08-08":33.395,"2021-08-07":33.422,"2021-08-06":33.45,"2021-08-05":33.263,"2021-08-04":33.137,"2021-08-03":33.022,"2021-08-02":32.957,"2021-08-01":32.895,"2021-07-31":32.887,"2021-07-30":32.887,"2021-07-29":32.87,"2021-07-28":32.82,"2021-07-27":32.937,"2021-07-26":32.929,"2021-07-25":32.94,"2021-07-24":32.963,"2021-07-23":32.928,"2021-07-22":32.887,"2021-07-21":32.857,"2021-07-20":32.829,"2021-07-19":32.854,"2021-07-18":32.807,"2021-07-17":32.795,"2021-07-16":32.783,"2021-07-15":32.686,"2021-07-14":32.605,"2021-07-13":32.654,"2021-07-12":32.664,"2021-07-11":32.506,"2021-07-10":32.482,"2021-07-09":32.512,"2021-07-08":32.534,"2021-07-07":32.334,"2021-07-06":32.271,"2021-07-05":32.112,"2021-07-04":32.136,"2021-07-03":32.125,"2021-07-02":32.126,"2021-07-01":32.073,"2021-06-30":32.029,"2021-06-29":32.108,"2021-06-28":31.918,"2021-06-27":31.785,"2021-06-26":31.816,"2021-06-25":31.816,"202

In [73]:
result_conversion_rate = r.json()
result_conversion_rate

{'conversion_rate': {'2021-08-12': 33.084,
  '2021-08-11': 33.145,
  '2021-08-10': 33.46,
  '2021-08-09': 33.464,
  '2021-08-08': 33.395,
  '2021-08-07': 33.422,
  '2021-08-06': 33.45,
  '2021-08-05': 33.263,
  '2021-08-04': 33.137,
  '2021-08-03': 33.022,
  '2021-08-02': 32.957,
  '2021-08-01': 32.895,
  '2021-07-31': 32.887,
  '2021-07-30': 32.887,
  '2021-07-29': 32.87,
  '2021-07-28': 32.82,
  '2021-07-27': 32.937,
  '2021-07-26': 32.929,
  '2021-07-25': 32.94,
  '2021-07-24': 32.963,
  '2021-07-23': 32.928,
  '2021-07-22': 32.887,
  '2021-07-21': 32.857,
  '2021-07-20': 32.829,
  '2021-07-19': 32.854,
  '2021-07-18': 32.807,
  '2021-07-17': 32.795,
  '2021-07-16': 32.783,
  '2021-07-15': 32.686,
  '2021-07-14': 32.605,
  '2021-07-13': 32.654,
  '2021-07-12': 32.664,
  '2021-07-11': 32.506,
  '2021-07-10': 32.482,
  '2021-07-09': 32.512,
  '2021-07-08': 32.534,
  '2021-07-07': 32.334,
  '2021-07-06': 32.271,
  '2021-07-05': 32.112,
  '2021-07-04': 32.136,
  '2021-07-03': 32.125,
  

In [84]:
# same method for check type
# assert isinstance(result_conversion_rate, dict) # assert is check type if not 'dict' is error
print(type(result_conversion_rate))

<class 'dict'>


In [135]:
conversion_rate = pd.DataFrame(result_conversion_rate)
conversion_rate = conversion_rate.reset_index().rename(columns={"index": "date"})
conversion_rate.head()

,date,conversion_rate
0,2021-04-01,31.194
1,2021-04-02,31.290
2,2021-04-03,31.256
3,2021-04-04,31.244
4,2021-04-05,31.342


### Join data
- transaction = `timestamp`
- conversion_rate = `date`

In [130]:
transaction.rename(columns = {'timestamp':'date'}, inplace = True)
transaction['date'].head(1)

0   2021-05-01 00:00:01
Name: date, dtype: datetime64[ns]

In [136]:
conversion_rate['date'].head(1)

0    2021-04-01
Name: date, dtype: object

In [150]:
# type 2 table do not match
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
transaction['date'].head(1)

0    2021-05-01
Name: date, dtype: object

In [151]:
final_df = transaction.merge(conversion_rate, how= 'left', left_on= 'date', right_on= 'date')
final_df.head(1)

,date,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,conversion_rate
0,2021-05-01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00,31.14


In [152]:
# column Price is type `string` and has symbol `$`
final_df["Price"] = final_df.apply(lambda x: x["Price"].replace("$",""), axis=1).astype(float)
final_df["Price"].head(1)

0    35.0
Name: Price, dtype: float64

In [156]:
# can do 3 method
#final_df['THBPrice'] = final_df['Price'] * final_df['conversion_rate']
#final_df["THBPrice"] = final_df.apply(lambda x: x["Price"] * x["conversion_rate"], axis=1)

def convert_rate(price, rate):
    return price * rate

final_df["THBPrice"] = convert_rate( final_df["Price"] , final_df["conversion_rate"] )

final_df.head()

,date,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,conversion_rate,THBPrice
0,2021-05-01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,31.14,1089.9000
1,2021-05-01,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,31.14,610.0326
2,2021-05-01,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,31.14,980.9100
3,2021-05-01,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,19.95,31.14,621.2430
4,2021-05-01,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,17.49,31.14,544.6386


In [155]:
final_df.dtypes

date                     object
user_id                  object
book_id                   int64
country                  object
Book Title               object
Book Subtitle            object
Book Author              object
Book Narrator            object
Audio Runtime            object
Audiobook_Type           object
Categories               object
Rating                   object
Total No. of Ratings    float64
Price                   float64
conversion_rate         float64
THBPrice                float64
dtype: object

### Save to CSV

In [157]:
final_df.to_csv("output.csv", index=False)